In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Translation

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/translation/intro_translation.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Flanguage%2Ftranslation%2Fintro_translation.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/translation/intro_translation.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/translation/intro_translation.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


---

* Author: Holt Skinner
* Created: Jan 2024

---

## Overview

This notebook demostrates how to use the [Google Cloud Translation API](https://cloud.google.com/translate) to translate text in [130+ languages](https://cloud.google.com/translate/docs/languages).

### Objective

This tutorial uses the following Google Cloud AI services and resources:

- [Cloud Translation API](https://cloud.google.com/translate/docs/overview)
- Cloud Storage


### Costs

This tutorial uses billable components of Google Cloud:

* Cloud Translation
* Cloud Storage

Learn about [Translate pricing](https://cloud.google.com/translate/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Getting Started


### Install Vertex AI SDK, other packages and their dependencies

Install the following packages required to execute this notebook.

In [2]:
# Install the packages
%pip install --user --upgrade -q google-cloud-aiplatform google-cloud-translate pandas tqdm


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Run the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

Set the project and region.

* Please note the **available regions** for Translation, see [documentation](https://cloud.google.com/translate/docs/advanced/endpoints)

In [6]:
PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}

TRANSLATE_LOCATION = "us"  # @param {type:"string"}
VERTEXAI_LOCATION = "us-central1"  # @param {type:"string"}

### Authenticating your notebook environment

* If you are using **Colab** to run this notebook, run the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

    ! gcloud config set project {PROJECT_ID}
    ! gcloud auth application-default login -q

Initialize the [Vertex AI SDK](https://cloud.google.com/vertex-ai/docs/python-sdk/use-vertex-ai-python-sdk)

In [ ]:
import vertexai

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=VERTEXAI_LOCATION)

### Import libraries

In [4]:
import json
import os
from pathlib import Path
from typing import Dict, List, Tuple

from tqdm import tqdm
import pandas as pd

from google.api_core.client_options import ClientOptions
from google.cloud import translate

from vertexai.preview.generative_models import GenerativeModel, GenerationConfig

### Create client

In [7]:
DEFAULT_LANGUAGE = "en"

# https://cloud.google.com/translate/docs/advanced/translate-text-advance
# https://cloud.google.com/python/docs/reference/translate/latest/google.cloud.translate_v3.services.translation_service.TranslationServiceClient#google_cloud_translate_v3_services_translation_service_TranslationServiceClient_translate_text
# https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/document_transformers/google_translate.py#L8

# self._client =
#         self._parent_path = self._client.common_location_path(project_id, location)
#         # For some reason, there's no `model_path()` method for the client.
#         self._model_path = (
#             f"{self._parent_path}/models/{model_id}" if model_id else None
#         )
#         self._glossary_path = (
#             self._client.glossary_path(project_id, location, glossary_id)
#             if glossary_id
#             else None
#         )

### Make API Call

In [ ]:


# response = translate_client.translate_text(
#     request=translate.TranslateTextRequest(
#         contents=[doc.page_content for doc in documents],
#         parent=self._parent_path,
#         model=self._model_path,
#         glossary_config=translate.TranslateTextGlossaryConfig(
#             glossary=self._glossary_path
#         ),
#         source_language_code=kwargs.get("source_language_code", None),
#         target_language_code=kwargs.get("target_language_code"),
#         mime_type=kwargs.get("mime_type", "text/plain"),
#     )
# )

#         # If using a glossary, the translations will be in `glossary_translations`.
#         translations = response.glossary_translations or response.translations

#          [
#             Document(
#                 page_content=translation.translated_text,
#                 metadata={
#                     **doc.metadata,
#                     "model": translation.model,
#                     "detected_language_code": translation.detected_language_code,
#                 },
#             )
#             for doc, translation in zip(documents, translations)
#         ]


# Initialize Translation client
def translate_text(
    text: str = "YOUR_TEXT_TO_TRANSLATE", project_id: str = "YOUR_PROJECT_ID", location: str = "global",
) -> translate.TranslateTextResponse:
    """Translating Text."""
    
    api_endpoint = f"translate-{TRANSLATE_LOCATION}.googleapis.com" if TRANSLATE_LOCATION != "global" else "translate.googleapis.com"    
    
    client = translate.TranslationServiceClient(client_options=ClientOptions(api_endpoint=api_endpoint))
    parent = client.common_location_path(project_id, location)

    # Translate text from English to French
    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        request={
            "parent": parent,
            "contents": [text],
            "mime_type": "text/plain",  # mime types: text/plain, text/html
            "source_language_code": "en-US",
            "target_language_code": "fr",
        }
    )

    # Display the translation for each input text provided
    for translation in response.translations:
        print(f"Translated text: {translation.translated_text}")

    return response